In [2]:
import configparser

Config = configparser.ConfigParser()

Config.read('settings.cfg')

def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                print("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1



dir_sanofi_share = ConfigSectionMap("SectionOne")['sanofi']
dir_local = ConfigSectionMap("SectionOne")['local']

print ("dir_sanofi_share is %s" % (dir_sanofi_share))
print ("dir_local is %s" % (dir_local))


dir_sanofi_share is C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/
dir_local is C:/Users/mark_/Documents/McLaren2021/Sanofi/


In [3]:
import pandas as pd
import numpy as np
import os
from glob import iglob

from datetime import datetime
import matplotlib.pyplot as plt



In [4]:
# function to read in files we need into a generic df
def create_df_from_file(indir, infolder, infilesearch):
    folder = infolder
    path = os.path.join(indir, folder)


    df = []
    df_created = pd.DataFrame()

    # do a recursive search for the files now we have multiple dirs
    for filename in iglob(path + '/**/*' + infilesearch + '*', recursive=True):
        df = pd.read_csv(filename)
        df_created = df_created.append(df)

    return df_created

In [5]:
folder = 'IP21_data'

# call function with dir, folder, search criteria to find files, name of dataframe to create
# these are the bad pen counts
df_ORDER = create_df_from_file(dir_sanofi_share, folder, "_ORDER")
df_ORDER['IP_TREND_TIME'] = pd.to_datetime(df_ORDER['IP_TREND_TIME'], format='%d-%b-%y %H:%M:%S.%f')

In [6]:
names = df_ORDER.Name.unique()
names

array(['36630901_ORDERNUMBER', '36630901_ZA_ORDERNUMBER',
       '36650901_ORDERNUMBER'], dtype=object)

In [7]:
df_ORDER.groupby('Name')['IP_TREND_VALUE'].describe()

,count,unique,top,freq
Name,,,,
36630901_ORDERNUMBER,45.0,39.0,80764309.0,3.0
36630901_ZA_ORDERNUMBER,45,13,1F004,8
36650901_ORDERNUMBER,83.0,48.0,80760793.0,7.0


In [31]:
df_36630901_ORDERNUMBER = df_ORDER[(df_ORDER.Name == '36630901_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)].copy()
df_36650901_ORDERNUMBER = df_ORDER[(df_ORDER.Name == '36650901_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)].copy()
df_36630901_ZA_ORDERNUMBER = df_ORDER[(df_ORDER.Name == '36630901_ZA_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)].copy()

df_36630901_ORDERNUMBER.rename(columns={'IP_TREND_VALUE':'36630901_ORDERNUMBER'}, inplace=True)
df_36650901_ORDERNUMBER.rename(columns={'IP_TREND_VALUE':'36650901_ORDERNUMBER'}, inplace=True)
df_36630901_ZA_ORDERNUMBER.rename(columns={'IP_TREND_VALUE':'36630901_ZA_ORDERNUMBER'}, inplace=True)

merge = pd.merge(df_36630901_ORDERNUMBER[['IP_TREND_TIME','36630901_ORDERNUMBER']], df_36630901_ZA_ORDERNUMBER[['IP_TREND_TIME','36630901_ZA_ORDERNUMBER']], on='IP_TREND_TIME')
merge2 = pd.merge(merge, df_36650901_ORDERNUMBER[['IP_TREND_TIME','36650901_ORDERNUMBER']], on='IP_TREND_TIME', how='outer')

In [34]:
merge2.sort_values('IP_TREND_TIME', inplace=True)
merge2

,IP_TREND_TIME,36630901_ORDERNUMBER,36630901_ZA_ORDERNUMBER,36650901_ORDERNUMBER
0,2021-08-02 20:43:03.700,80760843.0,1F022,NaN
45,2021-08-02 22:51:44.100,NaN,NaN,80757084.0
46,2021-08-03 19:56:16.900,NaN,NaN,80756656.0
47,2021-08-04 00:59:12.800,NaN,NaN,80756289.0
1,2021-08-04 06:46:41.500,80760846.0,1F022,NaN
...,...,...,...,...
121,2021-09-29 21:00:03.800,NaN,NaN,80763694.0
122,2021-09-29 23:19:26.600,NaN,NaN,80763694.0
123,2021-09-29 23:20:40.000,NaN,NaN,80763694.0
44,2021-09-30 12:40:19.400,80765625.0,1F004,NaN


In [13]:
df_ORDER.sort_values(['Name','IP_TREND_TIME'], inplace=True)
df_ORDER[(df_ORDER.Name == '36630901_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)]

,Name,IP_TREND_TIME,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS
22,36630901_ORDERNUMBER,2021-08-02 20:43:03.700,80760843.0,Good,Good
20,36630901_ORDERNUMBER,2021-08-04 06:46:41.500,80760846.0,Good,Good
18,36630901_ORDERNUMBER,2021-08-05 18:33:22.500,80760845.0,Good,Good
16,36630901_ORDERNUMBER,2021-08-06 00:01:26.600,80760847.0,Good,Good
12,36630901_ORDERNUMBER,2021-08-07 02:39:03.600,80761503.0,Good,Good
10,36630901_ORDERNUMBER,2021-08-10 10:03:15.800,80761506.0,Good,Good
8,36630901_ORDERNUMBER,2021-08-12 09:31:05.100,80759461.0,Good,Good
6,36630901_ORDERNUMBER,2021-08-12 17:09:20.200,80762175.0,Good,Good
4,36630901_ORDERNUMBER,2021-08-16 07:31:07.700,80762176.0,Good,Good
2,36630901_ORDERNUMBER,2021-08-16 07:33:03.300,80762176.0,Good,Good


In [16]:
df_ORDER[(df_ORDER.Name == '36630901_ZA_ORDERNUMBER') & (df_ORDER.IP_TREND_VALUE.isna() == False)]

,Name,IP_TREND_TIME,IP_TREND_VALUE,IP_TREND_QLEVEL,IP_TREND_QSTATUS
22,36630901_ZA_ORDERNUMBER,2021-08-02 20:43:03.700,1F022,Good,Good
20,36630901_ZA_ORDERNUMBER,2021-08-04 06:46:41.500,1F022,Good,Good
18,36630901_ZA_ORDERNUMBER,2021-08-05 18:33:22.500,1F022,Good,Good
16,36630901_ZA_ORDERNUMBER,2021-08-06 00:01:26.600,1F006,Good,Good
12,36630901_ZA_ORDERNUMBER,2021-08-07 02:39:03.600,1F006,Good,Good
10,36630901_ZA_ORDERNUMBER,2021-08-10 10:03:15.800,1F006,Good,Good
8,36630901_ZA_ORDERNUMBER,2021-08-12 09:31:05.100,0F001,Good,Good
6,36630901_ZA_ORDERNUMBER,2021-08-12 17:09:20.200,1F707,Good,Good
4,36630901_ZA_ORDERNUMBER,2021-08-16 07:31:07.700,1F709,Good,Good
2,36630901_ZA_ORDERNUMBER,2021-08-16 07:33:03.300,1F709,Good,Good
